In [1]:
! pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from transformers import (AutoModelForCausalLM, AutoTokenizer)
from datasets import load_dataset, Dataset
from typing import List, Dict, Optional
import torch

In [3]:
device = "cuda"
model = AutoModelForCausalLM.from_pretrained("../models/SOLAR-10-7B-v1-0", 
                                             local_files_only=True, 
                                             # torch_dtype=torch.float16, 
                                             device_map=device)
tokenizer = AutoTokenizer.from_pretrained("../models/SOLAR-10-7B-v1-0", 
                                          local_files_only=True)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

RuntimeError: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver.

In [40]:
MAX_EXAMPLES = 500
MAX_FEWSHOTS = 8

In [41]:
import pandas as pd    
jsonObj = pd.read_json(path_or_buf="../datasets/ASQA/dev.jsonl", lines=True)
dataset = Dataset.from_pandas(jsonObj)
dataset = dataset.select(range(MAX_EXAMPLES))

In [42]:
from prompts import (FEW_SHOT_EXAMPLES, PROMPT)

few_shot_examples = '\n\n'.join(FEW_SHOT_EXAMPLES[:MAX_FEWSHOTS])
prompts = [
    PROMPT.format(few_shot_examples = few_shot_examples,
                  question = data)
    for data in dataset['ambiguous_question']
]

In [3]:
model.num_parameters()

10731524096

In [4]:
text = "Explain the significance and applications of the formula a²+b²=c² in number theory."
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, 
                         max_new_tokens=64, 
                         do_sample=False, )
tokenizer.decode(outputs[0, inputs.input_ids.shape[1]:], pad_token_id = tokenizer.eos_token_id, skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

In [45]:
inputs = tokenizer(dataset['ambiguous_question'][0], return_tensors="pt").to(device)

In [44]:
dataset['ambiguous_question'][0]

'Who has the highest goals in world football?'

In [49]:
outputs = model.generate(**inputs, 
                         max_length=64, 
                         pad_token_id = tokenizer.eos_token_id, 
                         return_dict_in_generate=True, 
                         output_scores=True)

In [58]:
tokenizer.decode(outputs.sequences[0])

'<s> Who has the highest goals in world football?\nWho has the most goals in football in football in football in football in football in football in football in football in football in football in football in football in football in football in football in football in football in football in football in football in football in football in football in football'

In [50]:
outputs

GenerateDecoderOnlyOutput(sequences=tensor([[    1,  6526,   659,   272,  7881,  7661,   297,  1526,  6569, 28804,
            13, 11447,   659,   272,  1080,  7661,   297,  6569,   297,  6569,
           297,  6569,   297,  6569,   297,  6569,   297,  6569,   297,  6569,
           297,  6569,   297,  6569,   297,  6569,   297,  6569,   297,  6569,
           297,  6569,   297,  6569,   297,  6569,   297,  6569,   297,  6569,
           297,  6569,   297,  6569,   297,  6569,   297,  6569,   297,  6569,
           297,  6569,   297,  6569]], device='cuda:0'), scores=(tensor([[-12.4966,  -9.4589,   4.2763,  ...,  -7.0494,  -8.2579,  -7.0620]],
       device='cuda:0'), tensor([[-2.7341, -1.5249,  9.7068,  ..., -1.0406, -1.2222, -0.8011]],
       device='cuda:0'), tensor([[-7.4548,  0.6265,  7.3069,  ..., -9.7692, -8.8949, -6.2715]],
       device='cuda:0'), tensor([[-8.6624,  0.4954,  7.5608,  ..., -8.4176, -9.5677, -7.5875]],
       device='cuda:0'), tensor([[-7.3200,  0.8186,  7.7449,

In [52]:
inputs.input_ids.shape

torch.Size([1, 10])

In [61]:
for t in outputs.scores:
    values, indices = torch.sort(t)
    scores = values[0,-5:].flip(dims=[0])
    tokens = indices[0,-5:].flip(dims=[0])
    print(scores, tokens, [tokenizer.decode(tokens)])

tensor([14.4823,  9.3588,  9.3508,  9.3450,  9.0621], device='cuda:0') tensor([  13,  764,  342, 6526,  415], device='cuda:0') ['\n – | Who The']
tensor([15.4404, 14.2568, 14.1118, 13.9307, 13.6776], device='cuda:0') tensor([11447,  3195,  6479, 26703,  1014], device='cuda:0') ['WhoWhatfootWhichThe']
tensor([14.3441, 13.4724, 12.6928, 12.3255, 11.4236], device='cuda:0') tensor([  659,   349,    13, 13969,   460], device='cuda:0') ['has is\n scored are']
tensor([13.4148, 12.7340, 12.6190, 11.9187, 11.7300], device='cuda:0') tensor([  272, 13969,    13,  6526,  1080], device='cuda:0') ['the scored\n Who most']
tensor([13.0660, 12.5765, 12.5727, 12.1250, 11.6799], device='cuda:0') tensor([1080, 7881, 6526,   13, 4822], device='cuda:0') ['most highest Who\n Most']
tensor([13.0995, 11.3242, 11.1174, 10.7838,  9.8436], device='cuda:0') tensor([7661,   13, 6526, 6569, 3304], device='cuda:0') ['goals\n Who football World']
tensor([14.6844, 12.4059, 12.4013, 11.2102, 11.0514], device='cuda:0') 

In [10]:
output = tokenizer.batch_decode(output_ids.sequences, skip_special_tokens=True, clean_up_tokenization_spaces=False)
output[0]

'Explain the significance and applications of the formula a²+b²=c² in number theory.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

In [51]:
output_ids.sequences.shape

torch.Size([1, 64])

In [28]:
tokenizer.decode(output_ids,skip_special_tokens=True)

TypeError: argument 'ids': 'str' object cannot be interpreted as an integer

In [52]:
def logits_to_generate(logits, max_length=None):
    """
    Convert logits to model.generate-like output tensor using greedy decoding.
    
    Args:
        logits (torch.Tensor): Logits tensor of shape (batch_size, sequence_length, vocab_size).
        max_length (int): Maximum length of the generated sequence. Default is None.
    
    Returns:
        torch.Tensor: Generated tensor of shape (batch_size, sequence_length).
    """
    batch_size, sequence_length, vocab_size = logits.shape
    
    # Select the word with the highest logit value at each timestep
    generated = torch.argmax(logits, dim=-1)
    
    # Truncate to max_length
    if max_length is not None:
        generated = generated[:, :max_length]
    
    return generated

# 示例用法
logits = model(**inputs).logits
generated = logits_to_generate(logits)
print(tokenizer.decode(generated[0]))

The The
ain What difference of importance of the following for2 +b²=c². the theory.
abr>



In [34]:
out_logits = model(**inputs).logits

In [35]:
out_logits.shape

torch.Size([1, 28, 32000])

In [10]:
from tqdm import tqdm

def truncate(text: str, delimiters: List[str]) -> str:
    for d in delimiters:
        text = text.split(d)[0]
    return text

def generate(prompt: str, max_new_tokens:int,*args, **kwargs) -> str:
    '''Generate a single response'''
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output_ids = model.generate(input_ids, max_new_tokens = max_new_tokens, do_sample=True, **kwargs)
    input_length = input_ids.shape[1]
    decoded = tokenizer.batch_decode(output_ids[:, input_length:], skip_special_tokens=True)[0]
    return decoded

def generate_responses(prompts: List[str], 
                       max_new_tokens:int = 256, 
                       delimiters: List[str] = ['\n\n', tokenizer.eos_token], 
                       *args,
                       **kwargs) -> List[str]:
    '''
    Generate responses for a list of prompts. We don't use batch encode here.

    Args:
        prompts (List[str]): A list of prompts.
        max_new_tokens (int, optional): The maximum number of tokens to generate for each response. Defaults to 256.
        delimiters (List[str], optional): A list of delimiters used to truncate the generated response. Defaults to ['\n\n', tokenizer.eos_token].
        *args: Variable length argument list.
        **kwargs: Arbitrary keyword arguments.

    Returns:
        List[str]: A list of generated responses.
    '''
    responses = []
    for p in tqdm(prompts):
        r = generate(p, max_new_tokens, *args, **kwargs)
        responses.append(truncate(r, delimiters))
    return responses

In [11]:
responses = generate_responses(prompts[:1])

  0%|          | 0/1 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 1/1 [00:05<00:00,  5.35s/it]


In [33]:
inputs = tokenizer(prompts[0], return_tensors="pt").to(device)

tensor([[    1, 12628,   396,  ..., 16981, 28747, 28705]], device='cuda:0')

In [34]:
oids = model.generate(**inputs, max_new_tokens=256, )

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [40]:
inputs.input_ids.shape[1]

1765

In [43]:
oids

tensor([[    1, 12628,   396,  ...,    13,    13,    13]], device='cuda:0')

In [44]:
tokenizer.decode(oids[0],skip_special_tokens=True)

'Given an ambiguous question, figure out its interpretations and answer them one by one. \nQuestion: Who played bonnie in gone with the wind? \nAnswer: This question is ambiguous in terms of which version or adaptation of Gone with the Wind is being referred to. In order to figure out its interpretations, we need to consider different versions or adaptations of Gone with the Wind. Gone with the Wind has two versions or adaptations: the 1939 film Gone with the Wind or the 2008 musical Gone with the Wind. Therefore, this question has 2 interpretations: (1) Who played Bonnie in the 1939 film Gone with the Wind? (2) Who played Bonnie in the 2008 musical Gone with the Wind? The answers to all interpretations are: (1) The 1939 film Gone with the Wind’s character Bonnie was played by Eleanore Cammack "Cammie" King. (2) The 2008 musical Gone with the Wind’s character Bonnie was played by Leilah de Meza. \n\nGiven an ambiguous question, figure out its interpretations and answer them one by one.

In [ ]:
def extract_key_information(pred: str) -> str:
    '''Extract key information from the response.'''
    prefix_to_remove=['this question has 2 interpretations: (.*)$',
                      'this question has 1 interpretation: (.*)$']
    for pattern in prefix_to_remove:
        pred = pred.strip().split('\n\n', 1)[0].strip()
        find = re.compile(pattern).search(pred)
        if find:
            pred = find.group(1)
            break
    if find is None:
        logging.warning(f"Cannot extract key information from the response: {pred}")
    pred = re.sub(r'\(\d+\)\s', '', pred) # remove the index numbers
    return pred

In [ ]:
answers = [extract_key_information(r) for r in responses]

In [ ]:
dataset = dataset.add_column('responses', responses)
dataset = dataset.add_column('responses', responses)

In [ ]:
dataset.to_json('llama2-7b-chat')